# Grab Dimensions Data

https://app.dimensions.ai/discover/publication

This notebook handles the downloads of Dimensions data.


## Request Functions

This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [1]:
import requests
import json

from requests import ConnectionError, ReadTimeout

def grab_from_url_content(url):
    headers = {'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }
    rescontent = ''
    try:
        res = requests.get(url, headers=headers, timeout=10)
        rescontent = res.text
    except ConnectionError as ce:
        print('ConnectionError: ' + str(ce))
        return grab_from_url_content(url)
    except ReadTimeout as rte:
        print('ReadTimeout: ' + str(rte))
        return grab_from_url_content(url)

    return rescontent



## Parser

In [2]:
search_params = 'and_facet_for=2209&and_facet_for=2202&or_facet_source_title=jour.1082997&or_facet_for=2421'

In [3]:
from html.parser import HTMLParser

class DimensionsHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_next_trigger = False
    next_trigger_url = ''

    def handle_starttag(self, tag, attrs):
        if tag == 'article':
            self.in_article = True
        
        if self.in_article:
            for attr in attrs:
                if attr[0] == 'data-doc':
                    self.articles.append(json.loads(attr[1]))
                    break

        if tag == 'a':
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'nextPage-trigger'):
                    self.is_next_trigger = True
                if self.is_next_trigger and (attr[0] == 'href'):
                    self.next_trigger_url = 'https://app.dimensions.ai' + attr[1] + '&' + search_params
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_next_trigger = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = DimensionsHTMLParser()

## Grab Data

In [4]:
grab_url = 'https://app.dimensions.ai/discover/publication?' + search_params

In [5]:
parser.feed(grab_from_url_content(grab_url))

ConnectionError: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out.


In [6]:
next_trigger_anchor = ''
i = 0
while parser.next_trigger_url != next_trigger_anchor:
    if i % 10 == 0:
        print(str(i))
    i+=1
    next_trigger_anchor = parser.next_trigger_url
    parser.feed(grab_from_url_content(parser.next_trigger_url))

0


In [7]:
print(len(parser.articles))

129


In [8]:
with open("Journals/article_test.json", "w") as dump_f:
        dump_f.write(json.dumps(parser.articles))